# Create a Stage Volume Curve with PyQGIS

This Jupyter Notebook guides you to the process of using PyQGIS to create a stage volume curve from a DEM. The notebook follows the same code as used for the tutorial in QGIS, but can run without the QGIS interface.

## 1. Import libraries
Let's first import the necessary libraries for PyQGIS.

In [ ]:
from qgis.core import (
     QgsApplication, 
     QgsProcessingFeedback,
     QgsRasterLayer,
     QgsVectorLayer,
     QgsField,
     QgsVectorFileWriter
)

from qgis.PyQt.QtCore import QVariant

`QgsApplication` is needed to run QGIS and `QgsProcessingFeedback` is needed to run processing tools. We need `QgsRasterLayer` and `QgsVectorLayer` to work with raster and vector layers respectively. `QgsField` is needed manipulate fields in attribute tables. We use `QVariant` to set the data type of the fields. We need `QgsVectorFileWriter` to write the output to a `.csv` file.

## 2. Initialise QGIS without interface
The next step is to start QGIS without the interface.
The `setPrefixPath` needs to point to your environment. In this case it points to my user folder and the environment `tutorials` can be found under `Anaconda3/envs`.
The other two lines are needed to intialise QGIS. When you run the code ignore the depreciation warning that can show up.

In [ ]:
QgsApplication.setPrefixPath('C:/Users/hansa/Anaconda3/envs/tutorials', True)
qgs = QgsApplication([], False)
qgs.initQgis()

## 3. Load the DEM raster layer
Our DEM is stored in a GeoPackage in our `Data` folder. Let's define the folder (we need it later too, therefore we put it in a separate variable) and the dataset.

In [ ]:
projectPath = "./Data/"
inputRasterDEM = "GPKG:" + projectPath + "data_stagevolume.gpkg:DTM"

Now we can open the DEM raster layer using `QgsRasterLayer`, which needs the file name (`inputRasterDEM`), name of the layer `"DEM"` and we add `"gdal"`, because it's a GDAL supported format.

In [ ]:
demLayer = QgsRasterLayer(inputRasterDEM,"DEM","gdal")

## 4. Calculate band statistics to determine the range and increment
We need to determine the range of elevations first. With `bandStatistics` we can calculate statistics from the raster layer. Because we have a single band raster we use `bandstatistics(1)` to refer to band 1, the only band in our raster. We can now use `minimumValue` and `maximumValue` and print the results. [Here](https://opensourceoptions.com/blog/pyqgis-get-raster-band-statistics/) you can find info on the band statistics.

In [ ]:
stats = demLayer.dataProvider().bandStatistics(1)
demMinimum = stats.minimumValue
demMaximum = stats.maximumValue
print("min:",demMinimum,"m")
print("max:",demMaximum,"m")

What is the mean value of the raster? Try to write the code in the field below.

Instead of subtracting `demMinimum` from `demMaximum` we can also get the range directly by using `range`.

In [ ]:
demRange = stats.range
print("Elevation Difference:",demRange,"m")

The next step is to determine an increment of the level for which we want to have the corresponding volume later.
Let's use an increment of 10% of the range.

In [ ]:
increment = demRange / 10.0
print("Increment:",increment)

## 5. Iterate over elevation increments and calculate corresponding volumes
Now we can initialise the iteration over the increment. We need to set a counter `i=0` and create and empty list (`dbfList`) for the `.dbf` files that are created in each iteration.

In [ ]:
i = 0
dbfList = []

Python can not iterate with floats, therefore we define a function `frange` to deal with that:

In [ ]:
def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step

Now we can loop over the elevation range from the `demMinimum` to `demMaximum` with the `increment` as step size.

To run QGIS processing tools we need to first import `QgsNativeAlgorithms` from `qgis.analysis`, add them to the application and import `processing`.

Inside the loop we first create the output file name of the `.dbf` table.
Then we run the `rastersurfacevolume` tool by passing the correct variables to the dictionary. Remember that we get the dictionary of a processing tool from the history of the *Processing Toolbox* by first running the tool.
After running the tool we need to read the output `.dbf` table. Then we create another loop over the rows (features) in the table: `for feature in dbfTable.getFeatures():`.
For each row we convert the volume to the absolute value and from m3 to km3. The result of this calculation then needs to be stored in the table. We add two new fields with `addAtrributes`: *Level* and *VolAbsKm3*. Both have the data type Double, indicated with `QVariant.Double`. After defining the new fields we can add them to the table with `updateFields()`. Then we need to write the values of `level` and `VolumeKm3` to the corresponding fields. With `startEditing()` we toggle to editing mode. With another loop over the features we then assign the new values  `updateFeature` and save the changes with `commitChanges()`.

Add the end of the main loop we append the `dbf` files to the `dbfList` that we initialised before.


In [ ]:
from qgis.analysis import QgsNativeAlgorithms

QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())

import processing

for level in frange(demMinimum,demMaximum + 1,increment):
    # Define the output table name
    outTable = projectPath +"volume" + str(round(level*100.0))
    outTableDbf = outTable + ".dbf"
    
    # Run the raster surface volume tool with the variables
    Tool = processing.run("native:rastersurfacevolume", {'INPUT':demLayer,
                                                         'BAND':1,
                                                         'LEVEL':level,
                                                         'METHOD':1,
                                                         'OUTPUT_HTML_FILE':'TEMPORARY_OUTPUT',
                                                         'OUTPUT_TABLE':outTable + ".shp"})
    
    # Read the table
    dbfTable = QgsVectorLayer(outTableDbf, outTable, "ogr")
    
    # Convert the volumes from m3 to km3
    for feature in dbfTable.getFeatures():
        VolumeKm3 = abs(feature["Volume"])/1000000000.0
    
    # Add the level and volume in km3 fields to the table
    pr = dbfTable.dataProvider()
    pr.addAttributes([QgsField("Level", QVariant.Double),QgsField("VolAbsKm3", QVariant.Double)])
    dbfTable.updateFields()
    dbfTable.startEditing()
    for f in dbfTable.getFeatures():
        f["Level"] = level
        f["VolAbsKm3"] = VolumeKm3
        dbfTable.updateFeature(f)
    dbfTable.commitChanges()
 
    dbfList.append(outTableDbf)

## 6. Merge all dbf files into one stage volume table
Now the all `dbf` files created in the previous loop can be merged into one stage volume table. We use the processing tool `mergevectorlayers` and process the files in the `dbfList`. Note that the output is a shapefile. But shapefiles also have a `.dbf` file.

In [ ]:
# Merge all dbf files into one
processing.run("native:mergevectorlayers", {'LAYERS':dbfList,
                                            'CRS':None,
                                            'OUTPUT':projectPath + 'stagevolume.shp'})

## 7. Plot the stage volume curve in Python

The `stagevolume.dbf` file can be added to QGIS and used for plotting the stage volume curve with the Data Plotly plugin as described in the other tutorial. Here however we want to proceed with creating the curve without the QGIS GUI. Therefore it's better to convert the DBF file to CSV format.

First we read the DBF file with PyQGIS:

In [ ]:
vlayer = QgsVectorLayer(projectPath + 'stagevolume.dbf', "StageVolume", "ogr")

With `QgsVectorFileWriter.writeAsVectorFormat` we can convert this file to CSV format:

In [ ]:
QgsVectorFileWriter.writeAsVectorFormat(vlayer, "./Data/stagevolume.csv", "utf-8", vlayer.crs(), "CSV", layerOptions = ['GEOMETRY=AS_XYZ'])

With Pandas we can read the CSV and plot a line plot:

In [ ]:
import pandas as pd
%matplotlib inline

df  = pd.read_csv("./Data/stagevolume.csv")
df.plot(kind='line',x='VolAbsKm3',y='Level', title='Stage Volume Curve', xlabel='Volume (km3)', ylabel='Level (m above sea level)', legend=False)

*By __[Hans van der Kwast](http://www.linkedin.com/in/jvdkwast)__*<br>
*__[IHE Delft Institute for Water Education](http://www.un-ihe.org)__*<br>
*Twitter: @hansakwast*

*These materials are OpenCourseWare, licensed as [CC By-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)*